Another interpreter. We can represent the complete state with a dict; let's use a default dict so that we can initialise states to 0:

In [1]:
from collections import defaultdict
import re

We need a most recent value for the `rcv` instruction, so let's define a state as the triple `[R, S, C]`, where `R` is the most recent frequency, `S` is the ddict of the states, and `C` is the index of the current command. (List rather than tuple, so that we can take advantage of its mutability.)

So let's define a function which takes a state triple and a command, and updates the state accordingly.

In [2]:
def apply_command(command_str, stateIn):
    '''
    Updates stateIn with the command command_str
    '''
    
    # Useful to have a local function to check for ints:
    # (Could use exceptions, but I prefer not to if possible)
    def is_int(stringIn_str):
        '''
        True if stringIn_str can be converted to int
        '''
        if re.match('^\-?\d+$', stringIn_str):
            return True
        else:
            return False
    
    command_ls=command_str.strip().split()
    if command_ls[0]=='set':
        # If second argument is non-integral, treat it
        # as a ddict index
        if is_int(command_ls[2]):
            stateIn[1][command_ls[1]]=int(command_ls[2])
        else:
            stateIn[1][command_ls[1]]=stateIn[1][command_ls[2]]
        # Advance the command line
        stateIn[2]+=1
            
    elif command_ls[0]=='snd':
        # Play sound and set rcv to that frequency
        # PLAY SOUND!!
        if is_int(command_ls[1]):
            stateIn[0]=int(command_ls[1])
        else:
            stateIn[0]=stateIn[1][command_ls[1]]
        # Advance the command line
        stateIn[2]+=1
            
    elif command_ls[0]=='add':
        v=stateIn[1][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[1][command_ls[1]]=v+int(command_ls[2])
        else:
            stateIn[1][command_ls[1]]=v+stateIn[1][command_ls[2]]
        # Advance the command line
        stateIn[2]+=1

    elif command_ls[0]=='mul':
        v=stateIn[1][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[1][command_ls[1]]=v*int(command_ls[2])
        else:
            stateIn[1][command_ls[1]]=v*stateIn[1][command_ls[2]]
        # Advance the command line
        stateIn[2]+=1

    elif command_ls[0]=='mod':
        v=stateIn[1][command_ls[1]]
        if is_int(command_ls[2]):
            stateIn[1][command_ls[1]]=v%int(command_ls[2])
        else:
            stateIn[1][command_ls[1]]=v%stateIn[1][command_ls[2]]
        # Advance the command line
        stateIn[2]+=1

    elif command_ls[0]=='jgz':
        if is_int(command_ls[1]):
            v=int(command_ls[1])
        else:
            v=stateIn[1][command_ls[1]]
        # Jump if v>0
        if v>0:
            if is_int(command_ls[2]):
                stateIn[2]+=int(command_ls[2])
            else:
                stateIn[2]+=stateIn[1][command_ls[2]]
        # else just go forward 1:
        else:
            stateIn[2]+=1
    
    # Not clear what recovery is at the moment, so 
    # let's just print the output. Actually, we can
    # also return True in this case:
    elif command_ls[0]=='rcv':
        if is_int(command_ls[1]):
            v=int(command_ls[1])
        else:
            v=stateIn[1][command_ls[1]]
        # recover if v!=0
        if not v==0:
            print(stateIn[0])

            # Then advance 1 and return True:    
            stateIn[2]+=1
            return True
        stateIn[2]+=1

    else:
        raise ValueError("Command not recognised")
            
    return False

Set the test input:

In [3]:
testInput_str='''
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
'''

Let's try to apply the instructions:

In [4]:
state=[0, defaultdict(int), 0]
running_bl=True
commands_ls=testInput_str.strip().split('\n')

while running_bl:
    print(state)
    print(commands_ls[state[2]])
    t=apply_command(commands_ls[state[2]], state)
    if state[2]<0 or state[2]>=len(commands_ls) or t:
        running_bl=False

[0, defaultdict(<class 'int'>, {}), 0]
set a 1
[0, defaultdict(<class 'int'>, {'a': 1}), 1]
add a 2
[0, defaultdict(<class 'int'>, {'a': 3}), 2]
mul a a
[0, defaultdict(<class 'int'>, {'a': 9}), 3]
mod a 5
[0, defaultdict(<class 'int'>, {'a': 4}), 4]
snd a
[4, defaultdict(<class 'int'>, {'a': 4}), 5]
set a 0
[4, defaultdict(<class 'int'>, {'a': 0}), 6]
rcv a
[4, defaultdict(<class 'int'>, {'a': 0}), 7]
jgz a -1
[4, defaultdict(<class 'int'>, {'a': 0}), 8]
set a 1
[4, defaultdict(<class 'int'>, {'a': 1}), 9]
jgz a -2
[4, defaultdict(<class 'int'>, {'a': 1}), 7]
jgz a -1
[4, defaultdict(<class 'int'>, {'a': 1}), 6]
rcv a
4


Good! Now try with the puzzle input:

In [5]:
with open('data/day18.txt') as fIn:
    puzzleInput_str=fIn.read()

state=[0, defaultdict(int), 0]
running_bl=True
commands_ls=puzzleInput_str.strip().split('\n')

while running_bl:
    print(state)
    print(commands_ls[state[2]])
    t=apply_command(commands_ls[state[2]], state)
    if state[2]<0 or state[2]>=len(commands_ls) or t:
        running_bl=False

[0, defaultdict(<class 'int'>, {}), 0]
set i 31
[0, defaultdict(<class 'int'>, {'i': 31}), 1]
set a 1
[0, defaultdict(<class 'int'>, {'i': 31, 'a': 1}), 2]
mul p 17
[0, defaultdict(<class 'int'>, {'i': 31, 'a': 1, 'p': 0}), 3]
jgz p p
[0, defaultdict(<class 'int'>, {'i': 31, 'a': 1, 'p': 0}), 4]
mul a 2
[0, defaultdict(<class 'int'>, {'i': 31, 'a': 2, 'p': 0}), 5]
add i -1
[0, defaultdict(<class 'int'>, {'i': 30, 'a': 2, 'p': 0}), 6]
jgz i -2
[0, defaultdict(<class 'int'>, {'i': 30, 'a': 2, 'p': 0}), 4]
mul a 2
[0, defaultdict(<class 'int'>, {'i': 30, 'a': 4, 'p': 0}), 5]
add i -1
[0, defaultdict(<class 'int'>, {'i': 29, 'a': 4, 'p': 0}), 6]
jgz i -2
[0, defaultdict(<class 'int'>, {'i': 29, 'a': 4, 'p': 0}), 4]
mul a 2
[0, defaultdict(<class 'int'>, {'i': 29, 'a': 8, 'p': 0}), 5]
add i -1
[0, defaultdict(<class 'int'>, {'i': 28, 'a': 8, 'p': 0}), 6]
jgz i -2
[0, defaultdict(<class 'int'>, {'i': 28, 'a': 8, 'p': 0}), 4]
mul a 2
[0, defaultdict(<class 'int'>, {'i': 28, 'a': 16, 'p': 0}),

[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 947344688, 'b': 4688}), 10]
mul p 8505
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 8057166571440, 'b': 4688}), 11]
mod p a
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 1955411543, 'b': 4688}), 12]
mul p 129749
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 253712692292707, 'b': 4688}), 13]
add p 12345
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 253712692305052, 'b': 4688}), 14]
mod p a
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 384313884, 'b': 4688}), 15]
set b p
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 384313884, 'b': 384313884}), 16]
mod b 10000
[4688, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 384313884, 'b': 3884}), 17]
snd b
[3884, defaultdict(<class 'int'>, {'i': 40, 'a': 2147483647, 'p': 384313884, 'b': 3884}), 18]
add i -1
[3884, defaultdict(<class 'int'>, {'i': 39, '